<a href="https://colab.research.google.com/github/csrajath/CSC_8980_NLP/blob/main/csc_8980_NLP_exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: **Rajath Chikkatur Srinivasa** \
PantherID: **002552425**

In [1]:
# importing required libraries
import re, random, os, nltk,spacy, os
# from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import seaborn as sns
from nltk import bigrams, trigrams
# all sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
# importing models
#importing classifier types
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
#importing metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
# unzipping dataset
# !unzip /content/exam1_dataset.zip
# !tar -xvf /content/exam1_dataset.zip
!unzip /content/drive/MyDrive/Projects/NLP_Exam/exam1_dataset.zip

**Question 1) (20 points) Write a generic function that takes: Classification algorithm name, vectorization method name, training set with labels as parameters (total of 3 parameters should be passed). The function should take the classification algorithm name, the vectorization method’s name, and the training set and train the desired model. Use the default training parameters for the models we have seen in class. This function should return the trained model**

In [2]:
def generic_fun(algo_name, vec_method, train_set):
  # since the third parameter is a tuple, splitting the tuple values and assigning to respective variables
  x_train = train_set[0]
  y_train = train_set[1]
  if algo_name=='RandomForestClassifier' and vec_method=='CountVectorizer':
    rc_model = make_pipeline(CountVectorizer(), RandomForestClassifier())
    rc_model.fit(x_train, y_train)
    return rc_model
  elif algo_name=='RandomForestClassifier' and vec_method=='TfidfVectorizer':
    rt_model = make_pipeline(TfidfVectorizer(), RandomForestClassifier())
    rt_model.fit(x_train, y_train)
    return rt_model
  elif algo_name=='SVC' and vec_method=='CountVectorizer':
    sc_model = make_pipeline(CountVectorizer(), SVC())
    sc_model.fit(x_train, y_train)
    return sc_model
  elif algo_name=='SVC' and vec_method=='TfidfVectorizer':
    st_model = make_pipeline(TfidfVectorizer(), SVC())
    st_model.fit(x_train, y_train)
    return st_model
  elif algo_name=='MultinomialNB' and vec_method=='CountVectorizer':
    nc_model = make_pipeline(CountVectorizer(), MultinomialNB())
    nc_model.fit(x_train, y_train)
    return nc_model
  elif algo_name=='MultinomialNB' and vec_method=='TfidfVectorizer':
    nt_model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    nt_model.fit(x_train, y_train)
    return nt_model

**Question 2) (30 points) Using the function from question 1 to build the following models:
a) Model a: Naive Bayes, Vectorizer: TFIDF and Bag of Words, Training set should be 75% of the provided dataset. Leaving the remaining 25% for testing.
b) Model b: RandomForest, Vectorizer: TFIDF and Bag of Words, Training set should be 70% of the provided dataset. Leaving the remaining 30% for testing.
c) Model c: Support Vector Machines (SVC in sklearn), Vectorizer: TFIDF and Bag of Words, Training set should be 60% of the provided dataset. Leaving the remaining 40% for testing. NOTE: Set the random seed to: 12345. This needs to be consistently set to train the model AND split the data in test and train. If this is not done correctly, you will lose points as your answers will not be comparable with the grading key**

In [3]:
# creating set of documents i.e. list of all documents present in TRAINING folder
folder_path = '/content/exam1_dataset/TRAINING/'
file_content_corpus = []
target_label = []
# print(distance())
sub_folders = []
for i in os.walk(folder_path):
  sub_folders.append(i[0])
for folder in sub_folders:
  files = os.walk(folder).__next__()[-1]   
  for f in files:
    if not f.startswith('.'):                                                      
      file_content_corpus.append(open(os.path.join(folder, f),'r').read()) 
    # if os.path.join(folder,f) == '/content/exam1_dataset/TRAINING/negative/{}'.format(f)
    if folder=='/content/exam1_dataset/TRAINING/positive':
      target_label.append(1)
    elif folder=='/content/exam1_dataset/TRAINING/negative':
      target_label.append(0)

In [4]:
# Buuilding Model A:
np.random.seed(12345)
maX_train, maX_test, may_train, may_test = train_test_split(file_content_corpus, target_label, test_size=0.25)
mat = generic_fun('MultinomialNB', 'TfidfVectorizer', (maX_train, may_train)) # passing the training set as a tuple as it is single parameter in Q1 function
mac = generic_fun('MultinomialNB', 'CountVectorizer', (maX_train, may_train))

In [5]:
# Buuilding Model B:
np.random.seed(12345)
mbX_train, mbX_test, mby_train, mby_test = train_test_split(file_content_corpus, target_label,test_size=0.30)
mbt = generic_fun('RandomForestClassifier', 'TfidfVectorizer', (mbX_train, mby_train))
mac = generic_fun('RandomForestClassifier', 'CountVectorizer', (mbX_train, mby_train))

In [6]:
# Buuilding Model C:
np.random.seed(12345)
mcX_train, mcX_test, mcy_train, mcy_test = train_test_split(file_content_corpus, target_label, test_size=0.40)
mct = generic_fun('SVC', 'TfidfVectorizer', (mcX_train, mcy_train))
mcc = generic_fun('SVC', 'CountVectorizer', (mcX_train, mcy_train))

**Question 3) (30 points) Using the models from Question 2, evaluate each model with its respective training set (for model a, that set is 25% of the data, for model b, that set is 30% of the data, and for model c that set is 40% of the data. Be careful to not mix up the evaluation sets. With the predictions on the test set and show the following metrics: Accuracy, Precision, Recall, and Macro F1-score. With this in mind, please write and answer these questions in your notebook:
a) What model performs the best and why? (which metrics do you base this on, and why do you think it performs better than others).
b) Why is it important not to mix up the testing sets between different models? Think about this one.
c) Display in a single sorted dataframe (model name, training %, test %, accuracy, precision, recall, F1-score) all performance metrics, sorted by accuracy in descending
manner**

In [50]:
# predicting on modelA-CountVect
mac_predict = mac.predict(maX_test)
ma_name_cv = 'NaiveBayes-CV'
ma_name_tf = 'NaiveBayes-TF'
ma_name_train_per = '75%'
ma_name_test_per = '25%'
#  performance metrics for model A
ma_cv_acc = accuracy_score(may_test,mac_predict)
ma_cv_pre = precision_score(may_test,mac_predict)
ma_cv_re = recall_score(may_test,mac_predict)
ma_cv_f1 = f1_score(mac_predict, may_test, average='macro')
print('The performance metrics for ModelA-CountVect are: \n Accuracy:{}\n Precision={}\n Recall={}\n F1Score={}\n'.format(ma_cv_acc, ma_cv_pre, ma_cv_re, ma_cv_f1))
# predicting on modelA-TFIDF
mat_predict = mat.predict(maX_test)
#  performance metrics for model A
ma_tf_acc = accuracy_score(may_test,mat_predict)
ma_tf_pre = precision_score(may_test,mat_predict)
ma_tf_re = recall_score(may_test,mat_predict)
ma_tf_f1 = f1_score(mat_predict, may_test, average='macro')
print('The performance metrics for ModelA-TFIDF are: \n Accuracy:{}\n Precision={}\n Recall={}\n F1Score={}\n'.format(ma_tf_acc, ma_tf_pre, ma_tf_re, ma_tf_f1))

The performance metrics for ModelA-CountVect are: 
 Accuracy:0.84272
 Precision=0.8264984227129337
 Recall=0.858172289551261
 F1Score=0.8427170647229487

The performance metrics for ModelA-TFIDF are: 
 Accuracy:0.86368
 Precision=0.8645246770453793
 Recall=0.8548968227972487
 F1Score=0.8635693396725295



In [51]:
# predicting on modelB-CountVect
mbc_predict = mat.predict(mbX_test)
mb_name_cv = 'RandomForest-CV'
mb_name_tf = 'RandomForest-TF'
mb_name_train_per = '70%'
mb_name_test_per = '30%'
#  performance metrics for model B
mb_cv_acc = accuracy_score(mby_test,mbc_predict)
mb_cv_pre = precision_score(mby_test,mbc_predict)
mb_cv_re = recall_score(mby_test,mbc_predict)
mb_cv_f1 = f1_score(mbc_predict, mby_test, average='macro')
print('The performance metrics for ModelB-CountVect are: \n Accuracy:{}\n Precision={}\n Recall={}\n F1Score={}\n'.format(mb_cv_acc, mb_cv_pre, mb_cv_re, mb_cv_f1))
# predicting on modelB-TFIDF
mbt_predict = mat.predict(mbX_test)
#  performance metrics for model B
mb_tf_acc = accuracy_score(mby_test,mbt_predict)
mb_tf_pre = precision_score(mby_test,mbt_predict)
mb_tf_re = recall_score(mby_test,mbt_predict)
mb_tf_f1 = f1_score(mbt_predict, mby_test, average='macro')
print('The performance metrics for ModelB-TFIDF are: \n Accuracy:{}\n Precision={}\n Recall={}\n F1Score={}\n'.format(mb_tf_acc, mb_tf_pre, mb_tf_re, mb_tf_f1))

The performance metrics for ModelB-CountVect are: 
 Accuracy:0.8706666666666667
 Precision=0.8730684326710817
 Recall=0.8611867174741427
 F1Score=0.8705727795857461

The performance metrics for ModelB-TFIDF are: 
 Accuracy:0.8706666666666667
 Precision=0.8730684326710817
 Recall=0.8611867174741427
 F1Score=0.8705727795857461



In [52]:
# predicting on modelC-CountVect
mcc_predict = mat.predict(mcX_test)
mc_name_cv = 'SupportVectorMachine-CV'
mc_name_tf = 'SupportVectorMachine-TF'
mc_name_train_per = '60%'
mc_name_test_per = '40%'
#  performance metrics for model C
mc_cv_acc = accuracy_score(mcy_test,mcc_predict)
mc_cv_pre = precision_score(mcy_test,mcc_predict)
mc_cv_re = recall_score(mcy_test,mcc_predict)
mc_cv_f1 = f1_score(mcc_predict, mcy_test, average='macro')
print('The performance metrics for ModelC-CountVect are: \n Accuracy:{}\n Precision={}\n Recall={}\n F1Score={}\n'.format(mc_cv_acc, mc_cv_pre, mc_cv_re, mc_cv_f1))
# predicting on modelB-TFIDF
mct_predict = mat.predict(mcX_test)
#  performance metrics for model C

mc_tf_acc = accuracy_score(mcy_test,mct_predict)
mc_tf_pre = precision_score(mcy_test,mct_predict)
mc_tf_re = recall_score(mcy_test,mct_predict)
mc_tf_f1 = f1_score(mct_predict, mcy_test, average='macro')
print('The performance metrics for ModelC-TFIDF are: \n Accuracy:{}\n Precision={}\n Recall={}\n F1Score={}\n'.format(mc_tf_acc, mc_tf_pre, mc_tf_re, mc_tf_f1))

The performance metrics for ModelC-CountVect are: 
 Accuracy:0.8835
 Precision=0.8907441764584622
 Recall=0.8718724778046812
 F1Score=0.8834565887447416

The performance metrics for ModelC-TFIDF are: 
 Accuracy:0.8835
 Precision=0.8907441764584622
 Recall=0.8718724778046812
 F1Score=0.8834565887447416



a) What model performs the best and why? (which metrics do you base this on, and why do you think it performs better than others).
- SVM performs the best . By the defition of SVm is it largely used for binary classifications and this is a binary classification. On all the parameters SVM performs well. If there is one parameter to be called out then I'd base it on F1Score. Also, the test sample percentage for SVM is larger

b) Why is it important not to mix up the testing sets between different models? Think about this one.
- Each model has its own technique of predicting / classification. If we use the same testing set or mix-up the testing sets then it creates a bias to measure the model

c) Display in a single sorted dataframe (model name, training %, test %,accuracy, precision, recall, F1-score) all performance metrics, sorted by accuracy in descending manner

In [56]:
df1 = pd.DataFrame([ma_name_cv, ma_name_train_per, mc_name_test_per, ma_cv_acc, ma_cv_pre, ma_cv_re, ma_cv_f1]).transpose()
df2 = pd.DataFrame([ma_name_tf, ma_name_train_per, mc_name_test_per, ma_tf_acc, ma_tf_pre, ma_tf_re, ma_tf_f1]).transpose()
df3 = pd.DataFrame([mb_name_cv, mb_name_train_per, mc_name_test_per, mb_cv_acc, mb_cv_pre, mb_cv_re, mb_cv_f1]).transpose()
df4 = pd.DataFrame([mb_name_tf, mb_name_train_per, mc_name_test_per, mb_tf_acc, mb_tf_pre, mb_tf_re, mb_tf_f1]).transpose()
df5 = pd.DataFrame([mc_name_cv, mc_name_train_per, mc_name_test_per, mc_cv_acc, mc_cv_pre, mc_cv_re, mc_cv_f1]).transpose()
df6 = pd.DataFrame([mc_name_tf, mc_name_train_per, mc_name_test_per, mc_tf_acc, mc_tf_pre, mc_tf_re,mc_tf_f1]).transpose()
df = df1.append([df2,df3,df4,df5,df6])
df.columns = ['ModelName', 'trainPercentage', 'testPercentage','Accuracy', 'Precision', 'Recall', 'F1Score']
df.sort_values('Accuracy', ascending=False)

,ModelName,trainPercentage,testPercentage,Accuracy,Precision,Recall,F1Score
0,SupportVectorMachine-CV,60%,40%,0.8835,0.890744,0.871872,0.883457
0,SupportVectorMachine-TF,60%,40%,0.8835,0.890744,0.871872,0.883457
0,RandomForest-CV,70%,40%,0.870667,0.873068,0.861187,0.870573
0,RandomForest-TF,70%,40%,0.870667,0.873068,0.861187,0.870573
0,NaiveBayes-TF,75%,40%,0.86368,0.864525,0.854897,0.863569
0,NaiveBayes-CV,75%,40%,0.84272,0.826498,0.858172,0.842717


**Question 4) (15 points) Using the documents in the folder named UNLABELED, please use your best performing trained model from question 3 to predict their labels. Please do this individually for each document. Print to the screen the following items: Document Name, Predicted Label and using a text cell, write your own opinion if the label is correct and why - note you have to read the document to make your own opinion**

**Question 5) (20 points) Build a function that takes the set of documents as input and returns a cosine similarity matrix for those documents. Feed all documents in the TRAINING folder to this matrix. Instead of printing the returned cosine similarity matrix, create a heatmap plot from the returned matrix. Make sure your plot is nicely scaled, properly labeled, and uses a nice color range to show the similarity**

In [1]:
#driver code / function
def distance(docset):
  # vec = TfidfVectorizer(max_features=100)
  vec = TfidfVectorizer()
  X = vec.fit_transform(docset)
  cos_s = cosine_similarity(X, X)
  # cos_s = cosine_similarity(X, X, dense_output=False)
  # cos_s = np.around(cos_s, decimals=2)
  # cos_s = cos_s.astype(np.float16)
  cos_s = cos_s.astype(np.float32)
  return cos_s

In [ ]:
# function output stored in conf_mat variable
cos_mat = distance(file_content_corpus)

In [13]:
# #creating heatmap
# plt.figure(figsize=(6,7))
sns.heatmap(cos_mat, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

**Question 6) (15 points) Write a function that takes a cosine similarity matrix as input and returns a list with the top n document paris and their similarity. Note that you should only keep the document pairs that are unique and remove the comparisons of the document to itself. Print the top 50 similar document pairs. Compare the assigned class for each document and answer: Do all similar documents belong to the same class? Why or why not?**

In [1]:
def top_n_similar_works(cos_matrix, n):
  topNSimilar = []
  np.fill_diagonal(cos_matrix, 0)
  index_1d = cos_matrix.flatten().argsort()[-(2*n):]
  x_idx, y_idx = np.unravel_index(index_1d, cos_matrix.shape)
  for i in range(0,len(x_idx),2):
    x = x_idx[i]
    y = y_idx[i]
    if(x != y):
      topNSimilar.append((x,y,cos_matrix[x][y]))
  return topNSimilar[::-1]

In [ ]:
print('top 50 similar works\n')
for i in top_n_similar_works(cos_s,50):
  print(filenames[i[0]], 'is similar to', filenames[i[1]], '', i[2])

**Question 7) (20 points) Using Spacy’s part of speech tagger, process all sentences (hint: don’t forget to split the reviews) and count how many NOUN and VERB tags are found in all the movies review (TRAINING folder) separating them by label. In other words, how many NOUN and VERB tags are found in positive reviews, and how many NOUN and VERB tags are found in negative reviews. Answer the following question: When comparing both, do you see any differences? Why do you think about the differences? Or lack of them**

In [27]:
nlp = spacy.load("en_core_web_sm")
# total number of nouns and verbs in positive reviews
positive_reviews_files_path = "/content/exam1_dataset/TRAINING/positive/"
# v_count = []
# n_count = []
p_dict = {}
v_count = 0
n_count = 0
# p_count = 0
for i in os.listdir(positive_reviews_files_path):
  file_content = open(os.path.join(positive_reviews_files_path,i), 'r').read()
  doc = nlp(file_content)
  for i in doc:
    if i.pos_=='VERB':
      v_count+=1
    elif i.pos_=='NOUN':
      n_count+=1
    elif i.pos_ == 'PUNCT':
      if i in p_dict:
        p_dict[i.text]+=1
      else:
        p_dict[i.text]=1

In [28]:
print('Total number of NOUNs present in positive reviews are {}'.format(v_count))
print('Total number of VERBS present in positive reviews are {}'.format(n_count))

Total number of NOUNs present in positive reviews are 342125
Total number of VERBS present in positive reviews are 542978


In [29]:
# nlp = spacy.load("en_core_web_sm")
# total number of nouns and verbs in negative reviews
negative_reviews_files = "/content/exam1_dataset/TRAINING/negative/"
# v_count1 = []
# n_count1 = []
p_count1 = {}
v_count1 = 0
n_count1 = 0
count1 = 0
for i in os.listdir(negative_reviews_files):
  file_content = open(os.path.join(negative_reviews_files,i), 'r').read()
  doc = nlp(file_content)
  for i in doc:
    if i.pos_=='VERB':
      v_count1+=1
    elif i.pos_=='NOUN':
      n_count1+=1
    elif i.pos_ == 'PUNCT':
      if i in p_count1:
        p_count1[i.text]+=1
      else:
        p_count1[i.text]=1

In [30]:
print('Total number of NOUNs present in positive reviews are {}'.format(v_count1))
print('Total number of VERBS present in positive reviews are {}'.format(n_count1))

Total number of NOUNs present in positive reviews are 355559
Total number of VERBS present in positive reviews are 528475


- When comparing both, do you see any differences?
  - When I compare the count of NOUNs and VERBs  between POSITIVE and NEGATIVE folder - Yes there is a difference. When comparing them indivisually within each folder / dataset also there is a difference.
- Why do you think about the differences? Or lack of them
  - The difference is primarility because of the nature of the review. 

**Question 8) (20 points) Using the results from the PoS process in question 7, count how many different PUNCT tags are found and their respective counts from the full dataset provided (both negative and positives together). Using regex, write a set of regular expressions that generate the same counts from the dataset without using NLTK or Spacy, just regex. Can you get the same counts? If not, why do you think this is?**

In [45]:
total_punct = len(p_count1) + len(p_dict)
print('Total number of different PUNCT tags found is {}\n'.format(total_punct))
df = pd.DataFrame(p_dict.items(), columns=['PUNCT', 'Count'])
df1 = pd.DataFrame(p_count1.items(), columns=['PUNCT', 'Count'])
df_f = pd.concat([df, df1], ignore_index=True, sort=False)
df8 = df_f.groupby('PUNCT')['Count'].value_counts().to_frame()
df8.columns = ['Counts']
df8 = df8.reset_index().drop('Count', 1).drop_duplicates()
print('The respective counts from the full dataset for each punct is provided in the below dataframe\n')
df8

Total number of different PUNCT tags found is 8462

The respective counts from the full dataset for each punct is provided in the below dataframe



,PUNCT,Counts
0,!,2
1,!!!<br,1
2,!!!and,1
3,!.<br,1
4,!<br,2
...,...,...
6967,’s,2
6968,“,2
6969,”,2
6970,…,1


In [16]:
# Solving PUNT using RE
# merging the file contents to one string
file_content_corpus_merge = [' '.join(file_content_corpus[:])]
# the login used here for RE is that everything apart from word character and space is a PUNCT
punct_re = r'[^\w\s]'
reCount = re.findall(punct_re,file_content_corpus_merge[0])
print('{}'.format(len(reCount)))

1329527

- Can you get the same counts? If not, why do you think this is?
  - No, it is not the same count when done with regular expression and Spacy. Because, Spacy's coverage is both rule-based and contextual.

**Bonus Question: (40 points) Using the code from Class 09 - Word Embeddings, pre-tune BERT in order to classify movie reviews. You can use the full TRAINING folder for the tuning and use the UNLABELED folder for your final classification/prediction task. Do the label predictions from BERT match what your classifier from Question 4 predicted? If they don’t, are they better? Please say why. NOTE: This is a pro-level task and while you might be tempted to copy code from the Internet, do not do it, as you will fail the class if you are plagiarizing. The code from the class can be tweaked to do the task with some adjustments. This is definitely not easy, but that is why it is worth an extra 40 points!**

In [49]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

print('Tokenized: ', tokenizer.tokenize(sentences[0]))

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))